In [1]:
from server import LinkedViewsServer
data_folder = "/scratch/visual/bzfharth/in-silico-install-dir/project_src/in_silico_framework/getting_started/linked-views-example-data/backend_data_2023-06-22"

server = LinkedViewsServer(data_folder)
server.start(5000)
#server.stop()

start read: simulation_samples_original.csv
complete read
start read: simulation_samples.csv
complete read
server is running at port 5000
logs are written to /tmp/linked-views-server.log
set data:        server.set_data(vaex_df)
stop server:     server.stop()


In [2]:
import vaex

filename = "/scratch/visual/bzfharth/in-silico-install-dir/project_src/in_silico_framework/getting_started/linked-views-example-data/backend_data_2023-06-22/simulation_samples.csv"
df = vaex.from_csv(filename, copy_index=False)

server.set_data(df)




In [4]:
server.selections

{'global': {'columns': ['ephys.CaDynamics_E2_v2.apic.decay',
   'ephys.NaTa_t.axon.gNaTa_tbar'],
  'bin_ranges': [[[142.00789407637313, 143.80166708994],
    [1.4707900172377681, 1.5094689600303017]],
   [[142.00789407637313, 143.80166708994],
    [1.5094689600303017, 1.548147902822835]],
   [[143.80166708994, 145.59544010350692],
    [1.4707900172377681, 1.5094689600303017]],
   [[142.00789407637313, 143.80166708994],
    [1.3547531888601678, 1.393432131652701]],
   [[140.21412106280624, 142.00789407637313],
    [1.3547531888601678, 1.393432131652701]],
   [[138.42034804923935, 140.21412106280624],
    [1.3547531888601678, 1.393432131652701]],
   [[142.00789407637313, 143.80166708994],
    [1.4321110744452348, 1.4707900172377681]],
   [[143.80166708994, 145.59544010350692],
    [1.2000374176900337, 1.238716360482567]],
   [[143.80166708994, 145.59544010350692],
    [1.2773953032751006, 1.316074246067634]],
   [[143.80166708994, 145.59544010350692],
    [1.316074246067634, 1.3547531888

In [6]:
assert "global" in server.selections

columns = server.selections["global"]["columns"] # ['ephys.CaDynamics_E2_v2.apic.decay','ephys.NaTa_t.axon.gNaTa_tbar']
col1 = columns[0]
col2 = columns[1]

ranges = server.selections["global"]["bin_ranges"]

df.select_nothing(name="selection_global")
for idx, range_i in enumerate(ranges):    
    limit_i = [(range_i[0][0], range_i[0][1]), (range_i[1][0], range_i[1][1])]
    df.select_rectangle(df[col1], df[col2], limit_i, name="selection_global", mode="or")
    #print(limit_i)

df_selected_indices = df.evaluate(df["Unnamed: 0"], selection="selection_global")

print("num selected", df_selected_indices.shape)
print("indices", df_selected_indices)

(62,)
[ 201  203  254  309  485  512 1119 1374 1485 1542 1838 1883 2028 2286
 2331 2804 2977 3023 3322 3485 3764 3799 3983 4037 4474 4819 4922 4964
 5078 5636 5653 5714 5793 5815 5846 6033 6073 6114 6223 6237 6597 6667
 6697 7269 7336 7460 7588 7658 7743 7820 8228 8572 8598 8652 9143 9215
 9551 9564 9641 9678 9698 9709]


In [11]:
list(df.columns)

['Unnamed: 0',
 'sample_idx_original',
 'ephys.CaDynamics_E2_v2.apic.decay',
 'ephys.CaDynamics_E2_v2.apic.gamma',
 'ephys.CaDynamics_E2_v2.axon.decay',
 'ephys.CaDynamics_E2_v2.axon.gamma',
 'ephys.CaDynamics_E2_v2.soma.decay',
 'ephys.CaDynamics_E2_v2.soma.gamma',
 'ephys.Ca_HVA.apic.gCa_HVAbar',
 'ephys.Ca_HVA.axon.gCa_HVAbar',
 'ephys.Ca_HVA.soma.gCa_HVAbar',
 'ephys.Ca_LVAst.apic.gCa_LVAstbar',
 'ephys.Ca_LVAst.axon.gCa_LVAstbar',
 'ephys.Ca_LVAst.soma.gCa_LVAstbar',
 'ephys.Im.apic.gImbar',
 'ephys.K_Pst.axon.gK_Pstbar',
 'ephys.K_Pst.soma.gK_Pstbar',
 'ephys.K_Tst.axon.gK_Tstbar',
 'ephys.K_Tst.soma.gK_Tstbar',
 'ephys.NaTa_t.apic.gNaTa_tbar',
 'ephys.NaTa_t.axon.gNaTa_tbar',
 'ephys.NaTa_t.soma.gNaTa_tbar',
 'ephys.Nap_Et2.axon.gNap_Et2bar',
 'ephys.Nap_Et2.soma.gNap_Et2bar',
 'ephys.SK_E2.apic.gSK_E2bar',
 'ephys.SK_E2.axon.gSK_E2bar',
 'ephys.SK_E2.soma.gSK_E2bar',
 'ephys.SKv3_1.apic.gSKv3_1bar',
 'ephys.SKv3_1.apic.offset',
 'ephys.SKv3_1.apic.slope',
 'ephys.SKv3_1.axon.gS

In [3]:
df_col = df["ephys.CaDynamics_E2_v2.apic.decay"]
df_col2 = df["1BAC_caSpike_height.raw"]
binbycols = ["ephys.CaDynamics_E2_v2.apic.decay", "1BAC_caSpike_height.raw"]
value_col = "ephys.CaDynamics_E2_v2.apic.decay"

In [4]:
xycounts = df.count(binby=[df_col, df_col2], shape=(64, 128))
xycounts

array([[0, 0, 0, ..., 2, 1, 0],
       [0, 0, 0, ..., 1, 3, 2],
       [0, 0, 0, ..., 1, 5, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0]])

In [19]:
import requests

url = "http://127.0.0.1:5000/matrixServer/getDensity"

data = {
    "table" : "vaex_df",
    "format" : "mean",
    "columns" : [*binbycols,value_col],
    "density_grid_shape" : [10,10]
}

response = requests.post(url, json=data)
# Check the status code of the response
if response.status_code == 200:  # 200 means success
    data = response.json()  # Assuming the server returns JSON data
    print(data)
else:
    print(f"Request failed with status code: {response.status_code}")
    print(response.text)  # Print the response content, which may contain the error message from the server

{'columns': ['ephys.CaDynamics_E2_v2.apic.decay', '1BAC_caSpike_height.raw', 'ephys.CaDynamics_E2_v2.apic.decay'], 'indices': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], 'values': [[nan, 32.97851912445783, 28.783467174782846, 28.71127689678864, 30.010597916654607, 28.788098509866373, 28.595701630270245, 29.711063015579523, 29.430003409684378, 29.697886974632265], [38.466543830400845, 42.63694760751211, 48.64144449816129, 46.5368941155835, 47.718299316435036, 47.5777742341407, 47.33134256858265, 47.81419516454771, 47.77507350395298, 47.48092827107842], [62.71774504396427, 63.96235242408961, 61.35553800285296, 64.4744553148957, 65.27639028344669,

In [6]:
print(server.last_request)


b''


In [7]:
xycounts = df.min(df_col, binby=[df_col, df_col2], shape=(64, 128))
xycounts

array([[         inf,          inf,          inf, ...,  20.42735102,
         22.17325423,          inf],
       [         inf,          inf,          inf, ...,  24.72293007,
         23.1873414 ,  22.91251631],
       [         inf,          inf,          inf, ...,  27.89917967,
         26.91417087,  26.69122888],
       ...,
       [         inf,          inf,          inf, ...,          inf,
                 inf,          inf],
       [         inf,          inf,          inf, ...,          inf,
                 inf, 195.88171493],
       [         inf,          inf,          inf, ...,          inf,
        199.10768612,          inf]])

In [8]:
xycounts = df.max(df_col, binby=[df_col, df_col2], shape=(64, 128))
xycounts

array([[        -inf,         -inf,         -inf, ...,  21.99383864,
         22.17325423,         -inf],
       [        -inf,         -inf,         -inf, ...,  24.72293007,
         24.73608524,  22.97769683],
       [        -inf,         -inf,         -inf, ...,  27.89917967,
         27.90693979,  26.69122888],
       ...,
       [        -inf,         -inf,         -inf, ...,         -inf,
                -inf,         -inf],
       [        -inf,         -inf,         -inf, ...,         -inf,
                -inf, 195.88171493],
       [        -inf,         -inf,         -inf, ...,         -inf,
        199.10768612,         -inf]])

In [9]:
xycounts = df.mean(df_col, binby=[df_col, df_col2], shape=(64, 128))
xycounts

array([[         nan,          nan,          nan, ...,  21.21059483,
         22.17325423,          nan],
       [         nan,          nan,          nan, ...,  24.72293007,
         23.79077172,  22.94510657],
       [         nan,          nan,          nan, ...,  27.89917967,
         27.3576381 ,  26.69122888],
       ...,
       [         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       [         nan,          nan,          nan, ...,          nan,
                 nan, 195.88171493],
       [         nan,          nan,          nan, ...,          nan,
        199.10768612,          nan]])

In [10]:
xycounts = df.median_approx(df_col, binby=[df_col, df_col2], shape=(64, 128))
xycounts

array([[         nan,          nan,          nan, ...,  21.08648975,
         22.49337054,          nan],
       [         nan,          nan,          nan, ...,  24.60369174,
         23.19681094,  23.02095084],
       [         nan,          nan,          nan, ...,  28.12089372,
         27.18297319,  26.71401293],
       ...,
       [         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       [         nan,          nan,          nan, ...,          nan,
                 nan, 196.24314872],
       [         nan,          nan,          nan, ...,          nan,
        199.76035071,          nan]])

In [11]:
minmax = df.minmax(df_col)
print(minmax.tolist())

[20.03132915382484, 199.4086305105135]
